In [5]:
import matplotlib.pyplot as plt
import random as random
from math import dist



 #read file datapoint.txt, clean and store data in dictionary, return data dictionary
def generate_data_from_file():
    file_path = "C:\\Users\\NedyaIbrahim-AI23GBG\\Documents\\Github\\Python-Nedya-Ibrahim\\data\\datapoints.txt"
  
   #read file:
    with open(file_path, "r") as file_r: 
        datapoints = [row.strip("\n").split(", ") for row in file_r] #split each row into a list of 3 elements, remove newlines, and store list as element in datapoints list \n
        datapoints.remove(datapoints[0]) # remove file header
        datapoints = [[float(element) for element in point] for point in datapoints] # convert string value to float

 #datapoints = [[float(element) for element in point.split(", ")] for point in file_r.read().splitlines()[1:]]
    
    # dictionary containing all pokemon, ink their width/height measurments
    data = {
        "pikachu": {"width": [], "height": []},
        "pichu": {"width": [], "height": []}
}

    for i in datapoints:
        width, height, pokemon = float(i[0]), float(i[1]), int(i[2])
        if pokemon == 1:
            data["pikachu"]["width"].append(width)
            data["pikachu"]["height"].append(height)
        else:
            data["pichu"]["width"].append(width)
            data["pichu"]["height"].append(height)

    return data #return dict


In [6]:
 #read file testpoint.txt, clean and store data in dictionary, return data dictionary
def generate_test_data_from_file():
    file_path = "C:\\Users\\NedyaIbrahim-AI23GBG\\Documents\\Github\\Python-Nedya-Ibrahim\\data\\testpoints.txt"

    with open(file_path, "r") as file_r: # read file:
        testpoint = [row[4:-2].split(", ") for row in file_r] # slice and split each row into list of elements and store the in testpoints list
        testpoints.remove(testpoints[0]) # remove column header
        testpoints = [[float(element) for element in point] for point in testpoints] #convert string values to float
 # dictionary containing all undefined points, and their width/height measurements
    test_data = {
        "underfined" : {
        "width"     : [i[0] for i in testpoints],
        "height"    : [i[1] for i in testpoints]
        }
    }      

    return test_data # return dict



In [2]:
import matplotlib.pyplot as plt

#plot data from data (and test data)
#data from dictionary. Create scatterplot of data (test_data/incorrect_predictions)
def plot_data(data, test_data=None, incorrect_predictions=None):

    # Pikachus och Pichus datapunkter
    datasets = [
        (data["pikachu"], "yellow", "Pikachus"),
        (data["pichu"], "green", "Pichus")
    ]

    # Om testdata har lagts till som inmatningsparameter
    if test_data:
        undefined_data = test_data.get("undefined")
        if undefined_data:
            datasets.append((undefined_data, "red", "Undefined"))

        pikachu_test_data = test_data.get("pikachu")
        if pikachu_test_data:
            datasets.append((pikachu_test_data, "black", "Undefined (Pikachu)"))

        pichu_test_data = test_data.get("pichu")
        if pichu_test_data:
            datasets.append((pichu_test_data, "blue", "Undefined (Pichu)"))

    # Loopa igenom dataset och plotta dem
    for dataset, color, label in datasets:
        plt.scatter(dataset["width"], dataset["height"], color=color, label=label)

    # Skapa en lista med felaktiga förutsägelser och deras färger och etiketter
    if incorrect_predictions:
        incorrect_datasets = [
            (incorrect_predictions.get("pikachu"), "green", "Incorrect (actual Pikachu)"),
            (incorrect_predictions.get("pichu"), "purple", "Incorrect (actual Pichu)")
        ]

        # Loopa igenom felaktiga förutsägelser och plotta dem
        for dataset, color, label in incorrect_datasets:
            if dataset:
                plt.scatter(dataset["width"], dataset["height"], color=color, label=label)
 
    plt.title("Scatterplot of width/height measurements of pokemon")
    plt.xlabel("Width")
    plt.ylabel("Height")
    plt.legend()
    plt.show()
    
    

In [ ]:
file = open("C:\\Users\\NedyaIbrahim-AI23GBG\\Documents\\Github\\Python-Nedya-Ibrahim\\data\\testpoints.txt", "r")

content = file.readlines()


print(content)

file.close()